<a href="https://colab.research.google.com/github/csteele/nlp-disasters/blob/emma/nlp_disaster_tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Real or Not? NLP with Disaster Tweets
Twitter has become an important communication channel in times of emergency.The ubiquitousness of smartphones enables people to announce an emergency they’re observing in real-time. Because of this, more agencies are interested in programatically monitoring Twitter (i.e. disaster relief organizations and news agencies).

But, it’s not always clear whether a person’s words are actually announcing a disaster. In this competition you are given tweet data, some keywords and a location of the tweet and with this data the aim is to predict if the tweet is referencing a real disaster. 

*Link:* https://www.kaggle.com/c/nlp-getting-started




## What is this notebook?
It is a colloborative environment to share your code and progress with the team. There are a variety of ways to contribute:
 
1. Research NLP methods that we can try to include
2. Visulise and exploratore the data analysis to find trends and useful features
3. Build NLP models so that we can compare approaches and methods

## How to get started?
See this section for instructions on how to join kaggle and get your API key, how to version control, and how to use colabs.

**Step (1)** - Go to Kaggle and download your API key. This will be used to download the Kaggle dataset and manipulate it within your Colab environment.
![alt text](https://docs.google.com/uc?export=download&id=1gFbQABK3YlaHdI-u4hQ2emtuVtbQz_71)

**Step (2)** - Sign up to GitHub ([here](https://github.com)). This will be used to version control and share your work with the team. 

There is a master branch (notebook) that has the teams changes.  However, as we make our individual changes we will do it under a branch in our own name - these changes will not be visible in the master notebook. You can save in your own branch and when you are ready you can merge/copy those changes to the master branch for everyone to see. 

# Setup and Load Data  (Changed)
In this section we will set up and load the data.  The below section allows you to upload your personal kaggle API file that you downloaded in Setup Step (1). 

In [3]:
from google.colab import files
files.upload() #upload kaggle api key
!mkdir -p ~/.kaggle #create directory
!cp kaggle.json ~/.kaggle/ #copy uploaded file to new directory

Saving kaggle.json to kaggle.json


Now that we have our API key ready to use the next step downloads the training data and displays it. 

In [4]:
!kaggle competitions download -f train.csv --competition nlp-getting-started #download train.csv from compettition
import pandas as pd
train = pd.read_csv('train.csv')
train.head(5)

  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 63.8MB/s]


,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


## Imports

In [0]:
import plotly.express as px ## used for interactive plots


#Exploratory Data Analysis
In this section we can explore the data to answer questions that help us understand what the data contains and how it could be used to predict an actual disaster vs fake news / exagerations. 

## Data Quality

In [6]:
train_n = train.shape[0]
train_rate=train['target'].mean()
string = 'The training data contains {} samples with {}% tweets about real disasters.'
print(string.format(train_n,round(train_rate*100,2)))


The training data contains 7613 samples with 42.97% tweets about real disasters.


In [7]:
print('Data completeness per column')
train.count() / train_n


Data completeness per column


id          1.000000
keyword     0.991987
location    0.667280
text        1.000000
target      1.000000
dtype: float64

## Data Exploration


### Do some locations have a higher proportion of disasters?

In [7]:
location_filter = 10

location=train.groupby('location').agg(['mean','count'])['target'].reset_index()
location=location.loc[location['count']>location_filter].sort_values(by='mean',ascending=False).head(25)
px.bar(location,x='location',y='mean', hover_data=["location","mean","count"])

**Key Word Frequency**

In [102]:
train.keyword.nunique()
df = train.groupby('keyword')['id'].nunique()
print(df)

keyword
ablaze                 36
accident               35
aftershock             34
airplane%20accident    35
ambulance              38
                       ..
wounded                37
wounds                 33
wreck                  37
wreckage               39
wrecked                39
Name: id, Length: 221, dtype: int64


In [0]:
import nltk
freq = nltk.FreqDist(train.keyword)
for key,val in freq.items():
    print(str(key) + ':' + str(val))
freq.plot(30, cumulative=False)

In [138]:
# Group by to calculate how many 0's and 1's per Keyword
Proportion=train.groupby(['keyword','target'])['target'].count()
# Unstack to pivot values
Proportion=Proportion.unstack()
# Clause to chnage NaN to either 0 or 1
# = Proportion.where(pd.notnull(Proportion[['0', '1']]), 0, 1)
# Add column - Total number of texts
Proportion['TotalCount']=Proportion[0]+Proportion[1]
# Add column - Percent of texts per keyword that are disasters
Proportion['ProportionDisaster']=Proportion[1]/Proportion['TotalCount']
print(Proportion)
# Scatter plot
px.scatter(Proportion,x="TotalCount",y="ProportionDisaster")


target                  0     1  TotalCount  ProportionDisaster
keyword                                                        
ablaze               23.0  13.0        36.0            0.361111
accident             11.0  24.0        35.0            0.685714
aftershock           34.0   NaN         NaN                 NaN
airplane%20accident   5.0  30.0        35.0            0.857143
ambulance            18.0  20.0        38.0            0.526316
...                   ...   ...         ...                 ...
wounded              11.0  26.0        37.0            0.702703
wounds               23.0  10.0        33.0            0.303030
wreck                30.0   7.0        37.0            0.189189
wreckage              NaN  39.0         NaN                 NaN
wrecked              36.0   3.0        39.0            0.076923

[221 rows x 4 columns]


# Research

## Bag of Words


# Model